<a href="https://www.kaggle.com/code/maimoonakhilji/scrap-google-play-store-applications?scriptVersionId=154958764" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Social apps data

1. Applications Data of category
2. Review of these applications


#### Install google-play-scraper

In [1]:
!pip install google-play-scraper


#### Import Necessory Libraries

In [2]:
import csv
from google_play_scraper import app, search, Sort, reviews_all,reviews
import pandas as pd

    The given output will help you know the data that we can have using google-play-scraper library.
    it retrieves data for one appliction in Social category and then returns the column name using .keys() function


In [3]:
results = search("Social", lang="en", country="us", n_hits=1)
for app_info in results:
    a = app_info
    print(app_info.keys())


dict_keys(['appId', 'icon', 'screenshots', 'title', 'score', 'genre', 'price', 'free', 'currency', 'video', 'videoImage', 'description', 'descriptionHTML', 'developer', 'installs'])


#### Scrap Top Applications Data

In [4]:
def Scrap_apps(category):
    
    results = search(category, lang="en", country="us")  # Adjust parameters as needed

    # Sort the results by the number of downloads in descending order
    results.sort(key=lambda x: int(x['installs'].replace(',', '').replace('+', '')), reverse=True)

    # Select the top 30 apps
    results = results[:30]

    # List to store app information
    app_data = []

    # Iterate through the search results and get detailed information for each app
    for app_info in results:
        app_id = app_info['appId']
        
        # Append app information to the list
        app_data.append({
             "AppID": app_id,
        "AppName": app_info['title'],
        "developer_name": app_info["developer"],
        "genre": app_info["genre"],
        "Rating": app_info['score'],
        "NumberOfDownloads": app_info['installs'],
        "Price": app_info.get('price', 'Free'),  # Assuming free if price is not available
        "Description": app_info['description']

        })
 
    df = pd.DataFrame(app_data)                # Save the app data to a CSV file
    df.to_csv("Apps_data.csv")
    return "Apps_data.csv"

#### Scrap Reviews of Applications

In [5]:
def get_review_data(app_data):
    
    desired_keys = ['reviewId', 'userName', 'content', 'score',"at", "reviewCreatedVersion" ,"appVersion"]
    col = ['Id', "AppName",'reviewId', 'userName', 'content', 'score',"at", "reviewCreatedVersion" ,"appVersion"]

    with open("reviews.csv", 'w',encoding='utf-8') as file:    # Open the file in write mode to clear its content
        csv_writer = csv.DictWriter(file, fieldnames=col)
        # Write the header row to the CSV file
        csv_writer.writeheader()
        pass
    
    with open(app_data, 'r',encoding='utf-8') as input_file: # Create CSV reader and writer objects
        csv_reader = csv.reader(input_file)
        header = next(csv_reader)
        
        # Process each row in the input CSV file
        for row in csv_reader:
            app_id = row[1]  # Assuming the app ID is in the first column
            app_name=row[2]

            # Fetch the reviews for the app
            app_reviews, continuation_token = reviews(app_id, sort='newest')
            
            
                  # Open the output CSV file
            with open("reviews.csv", 'a', newline='',encoding='utf-8') as output_file:

                csv_writer = csv.DictWriter(output_file, fieldnames=col)

                for review in app_reviews:
                     # Extract the desired keys from the review
                    review_data = {key: review[key] for key in desired_keys}

                  # Write the review data to the CSV file
                    csv_writer.writerow({**{'Id': app_id}, **{"AppName": app_name}, **review_data})
                    
    return "reviews.csv"


In [6]:
def main(category_name):
    
    apps = Scrap_apps(category_name)
    print("Applications data saved to Apps_data.csv.")
    
    get_review_data(apps)
    print("Reviews retrieval completed and saved to reviews.csv.")


In [7]:
main("Social")

Applications data saved to Apps_data.csv.
Reviews retrieval completed and saved to reviews.csv.


#### Read Apps data

In [8]:
applications = pd.read_csv("Apps_data.csv")
applications.head()

,Unnamed: 0,AppID,AppName,developer_name,genre,Rating,NumberOfDownloads,Price,Description
0,0,com.instagram.android,Instagram,Instagram,Social,3.994447,"5,000,000,000+",0,Little moments lead to big friendships. Share ...
1,1,com.facebook.katana,Facebook,"Meta Platforms, Inc.",Social,3.511102,"5,000,000,000+",0,Whether you’re looking for a spark of inspirat...
2,2,com.snapchat.android,Snapchat,Snap Inc,Communication,4.073230,"1,000,000,000+",0,Snapchat is a fast and fun way to share the mo...
3,3,com.twitter.android,X,X Corp.,Social,3.825432,"1,000,000,000+",0,The X app is the trusted global digital town s...
4,4,com.zhiliaoapp.musically,TikTok,TikTok Pte. Ltd.,Social,4.301375,"1,000,000,000+",0,TikTok is THE destination for mobile videos. O...


The given applications.shape()  returns the number of rows and column respectively. rows represents the application names while columns refer to details of each app.

In [9]:
applications.shape

(30, 9)

#### Read Reviews data

In [10]:
reviews = pd.read_csv("reviews.csv")
reviews.head()

,Id,AppName,reviewId,userName,content,score,at,reviewCreatedVersion,appVersion
0,com.instagram.android,Instagram,563e37fb-e87b-417c-9e85-98802fe31301,Anthony Copter,User Un-Friendly and Frustrating Only allows p...,1,2023-11-14 11:51:28,308.0.0.36.109,308.0.0.36.109
1,com.instagram.android,Instagram,d6043746-56a5-4a00-a6c6-3550d246602c,Caroline,Obviously a great platform but the updates and...,3,2023-11-14 12:18:12,308.0.0.36.109,308.0.0.36.109
2,com.instagram.android,Instagram,ae5fc099-4efa-4b54-a36a-d27de2bf010d,Lou,"Dark mode is broken, if i click a link, click ...",3,2023-11-16 04:12:23,308.0.0.36.109,308.0.0.36.109
3,com.instagram.android,Instagram,d5346f4f-8512-4faf-87bb-855faa0f5acd,P. J. Bovio,This app is abysmal!! It's taken a turn for th...,1,2023-11-03 07:59:24,307.0.0.34.111,307.0.0.34.111
4,com.instagram.android,Instagram,bd0d8a9f-57fd-45a6-9947-9ff8087e7ffb,AN - Alechia Neathery,"The app and website aren't loading, and I try ...",1,2023-09-20 13:59:43,301.1.0.33.110,301.1.0.33.110


In [11]:
reviews.shape

(3000, 9)

The given unique() returns the unique values.

In [12]:
reviews['AppName'].unique()

array(['Instagram', 'Facebook', 'Snapchat', 'X', 'TikTok',
       'Facebook Lite', 'LinkedIn: Jobs & Business News', 'Pinterest',
       'Likee - Short Video Community', 'Bigo Live - Live Streaming App',
       'Reddit', 'VK: music, video, messenger',
       'Twitch: Live Game Streaming', 'Tumblr—Fandom, Art, Chaos',
       'ZEPETO: Avatar, Connect & Play', 'Tango- Live Stream, Video Chat',
       'Threads, an Instagram app', 'OK: Social Network',
       'ShareChat Trends Videos & Live', 'Litmatch—Make new friends',
       'JAUMO Dating App: Chat & Date', 'Meetup: Social Events & Groups',
       'BeReal. Your friends for real.', 'Yubo: Make new friends',
       'Clubhouse', 'Weverse', 'Nextdoor: Neighborhood network',
       'Boo: Dating. Friends. Chat.', 'Plato - Games & Group Chats',
       'Truth Social'], dtype=object)

The .value_counts() returns the count of reviews scrapped for each application.

In [13]:
reviews['AppName'].value_counts()

AppName
Instagram                         100
Facebook                          100
Plato - Games & Group Chats       100
Boo: Dating. Friends. Chat.       100
Nextdoor: Neighborhood network    100
Weverse                           100
Clubhouse                         100
Yubo: Make new friends            100
BeReal. Your friends for real.    100
Meetup: Social Events & Groups    100
JAUMO Dating App: Chat & Date     100
Litmatch—Make new friends         100
ShareChat Trends Videos & Live    100
OK: Social Network                100
Threads, an Instagram app         100
Tango- Live Stream, Video Chat    100
ZEPETO: Avatar, Connect & Play    100
Tumblr—Fandom, Art, Chaos         100
Twitch: Live Game Streaming       100
VK: music, video, messenger       100
Reddit                            100
Bigo Live - Live Streaming App    100
Likee - Short Video Community     100
Pinterest                         100
LinkedIn: Jobs & Business News    100
Facebook Lite                     100
TikT